In [27]:
import torch
import numpy as np
%load_ext autoreload
%autoreload 2
import espaloma as esp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
torch.__version__

'1.4.0'

In [29]:
g = esp.Graph('c1ccccc1')

forcefield = esp.graphs.legacy_force_field.LegacyForceField(
    'smirnoff99Frosst'
)

g = forcefield.parametrize(g)

from espaloma.data.md import MoleculeVacuumSimulation

simulation = MoleculeVacuumSimulation(
        n_samples=10, n_steps_per_sample=10
)

g = simulation.run(g)

# layer
layer = esp.nn.layers.dgl_legacy.gn()

# representation
representation = esp.nn.Sequential(layer, [32, 'tanh', 32, 'tanh', 32, 'tanh'])

readout = esp.nn.readout.janossy.JanossyPooling(
    in_features=32, config=[32, 'tanh'],
)


net = torch.nn.Sequential(
        representation, 
        readout,
        esp.mm.geometry.GeometryInGraph(),
        esp.mm.energy.EnergyInGraph(),
        esp.mm.energy.EnergyInGraph(suffix='_ref'),
)


net(g.heterograph)


/Users/wangy1/anaconda3/envs/pinot/lib/python3.7/site-packages/dgl/base.py:25: UserWarning: Currently adjacency_matrix() returns a matrix with destination as rows by default.  In 0.5 the result will have source as rows (i.e. transpose=True)
  warnings.warn(msg, warn_type)
/Users/wangy1/anaconda3/envs/pinot/lib/python3.7/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Graph(num_nodes={'n1': 12, 'n2': 24, 'n3': 36, 'n4': 48, 'nonbonded': 6, 'onefour': 48, 'g': 1},
      num_edges={('n1', 'n1_neighbors_n1', 'n1'): 24, ('n1', 'n1_as_0_in_n2', 'n2'): 24, ('n2', 'n2_has_0_n1', 'n1'): 24, ('n1', 'n1_as_1_in_n2', 'n2'): 24, ('n2', 'n2_has_1_n1', 'n1'): 24, ('n1', 'n1_as_0_in_n3', 'n3'): 36, ('n3', 'n3_has_0_n1', 'n1'): 36, ('n1', 'n1_as_1_in_n3', 'n3'): 36, ('n3', 'n3_has_1_n1', 'n1'): 36, ('n1', 'n1_as_2_in_n3', 'n3'): 36, ('n3', 'n3_has_2_n1', 'n1'): 36, ('n1', 'n1_as_0_in_n4', 'n4'): 48, ('n4', 'n4_has_0_n1', 'n1'): 48, ('n1', 'n1_as_1_in_n4', 'n4'): 48, ('n4', 'n4_has_1_n1', 'n1'): 48, ('n1', 'n1_as_2_in_n4', 'n4'): 48, ('n4', 'n4_has_2_n1', 'n1'): 48, ('n1', 'n1_as_3_in_n4', 'n4'): 48, ('n4', 'n4_has_3_n1', 'n1'): 48, ('n2', 'n2_as_0_in_n3', 'n3'): 36, ('n3', 'n3_has_0_n2', 'n2'): 36, ('n2', 'n2_as_1_in_n3', 'n3'): 36, ('n3', 'n3_has_1_n2', 'n2'): 36, ('n2', 'n2_as_0_in_n4', 'n4'): 48, ('n4', 'n4_has_0_n2', 'n2'): 48, ('n2', 'n2_as_1_in_n4', 'n4'): 48

In [30]:
force = torch.autograd.grad(
    g.nodes['n2'].data['u'].sum(),
    g.nodes['n1'].data['xyz'],
    create_graph=True,
    retain_graph=True,
)[0]


In [31]:
print(force.grad_fn)

None
